In [ ]:
CONFIG_FILE_NAME = 'sbwatcher.json'
CONFIG_FOLDER_NAME = 'configs'

import os
import json
import datetime

from mysql import connector as connector

%matplotlib inline
import matplotlib.pyplot as plt

dirName, _ = os.path.split(os.path.abspath(''))
config_file_path = dirName + os.path.sep + os.path.sep + CONFIG_FOLDER_NAME + os.path.sep + CONFIG_FILE_NAME

config_file = open(config_file_path, 'r+')
configs = json.load(config_file)
config_file.close()

connect = connector.connect(**configs['mysql'])

In [ ]:
start_date = datetime.datetime.strptime('2020.1.1', '%Y.%m.%d')
end_date = datetime.datetime.strptime('2020.10.31', '%Y.%m.%d')

In [ ]:
query = """
SELECT buy_price, sell_price, event_ts 
FROM f_rates WHERE fin_id = 1 and event_ts BETWEEN {0} AND {1} ORDER BY event_ts;
""".format(start_date.timestamp(), end_date.timestamp())

cursor = connect.cursor(dictionary=True)
cursor.execute(query) 
rates = cursor.fetchall()

In [ ]:
rate_range = [datetime.datetime.fromtimestamp(item['event_ts']) for item in rates]
sells_data = [item['sell_price'] for item in rates]
buys_data = [item['buy_price'] for item in rates]


r_range = rate_range  + [rate_range[-1]] + [rate_range[0]]

spred_data = [item['sell_price'] - item['buy_price'] for item in rates]
min_spred = min(spred_data)
spred_data = spred_data + [min_spred] + [min_spred]

fig, ax1 = plt.subplots(figsize=(30, 6))

ax1.fill(
    r_range , 
    spred_data,
    color = 'green', 
    alpha = 0.25

)

ax2 = ax1.twinx()

ax2.plot(
    rate_range, 
    sells_data,
)
ax2.plot(
    rate_range, 
    buys_data
)


plt.show()